In [1]:
import torch
from torch import nn
from sequitur.models import LINEAR_AE, LSTM_AE
from sequitur import quick_train
from sequitur.quick_train import train_model
import yfinance as yf
import pandas as pd
from finta import TA
import numpy as np
from typing import List, Tuple, Dict, Union, Optional

In [2]:
train_seqs = [torch.randn(4) for _ in range(100)] # 100 sequences of length 4

In [3]:
encoder, decoder, _, _ = quick_train(LINEAR_AE, train_seqs, encoding_dim=2, denoise=True)

C:\Users\Maciej\anaconda3\envs\pytorch_gpu\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [4]:
encoder(torch.randn(4))

tensor([-0.9200, -0.3487], grad_fn=<TanhBackward0>)

In [5]:
encoder(torch.randn(2, 4))

tensor([[ 0.8829, -0.1512],
        [ 0.2971,  0.7620]], grad_fn=<TanhBackward0>)

### Sequence of 1D vectors (perfect for our purposes)

In [6]:
model = LSTM_AE(
  input_dim=3,
  encoding_dim=7,
  h_dims=[64],
  h_activ=None,
  out_activ=None
)

In [7]:
x = torch.randn(10, 3) # Sequence of 10 3D vectors
z = model.encoder(x) # z.shape = [7]

In [8]:
x_prime = model.decoder(z, seq_len=10) # x_prime.shape = [10, 3]

In [9]:
x_prime

tensor([[0.1040, 0.1869, 0.1644],
        [0.1417, 0.2564, 0.2296],
        [0.1515, 0.2775, 0.2519],
        [0.1514, 0.2804, 0.2570],
        [0.1486, 0.2773, 0.2558],
        [0.1459, 0.2731, 0.2530],
        [0.1438, 0.2693, 0.2503],
        [0.1424, 0.2664, 0.2481],
        [0.1415, 0.2642, 0.2466],
        [0.1409, 0.2627, 0.2455]], grad_fn=<MmBackward0>)

### Trying it out on real data

In [10]:
def create_ta_features(ticker='^GSPC', start_='2010-01-01', end_='2022-12-31', interval_='1d', fillna=True, scale_to_std=True, fill_weekends=True):
    """
    Creates dataframe with technical analysis features
    :param ticker: ticker symbol to download data for (default is S&P 500)
    :param start_: start date
    :param end_: end date
    :param interval_: data frequency
    :param fillna: whether to fill in missing values
    :param scale_to_std: whether to scale to standard deviation
    :param fill_weekends: whether to fill in weekends
    :return: dataframe with technical analysis features
    """
    # download data
    df = yf.download(ticker, start_, end_, interval=interval_)
    # rename columns
    df.rename(columns={"Open": "open", "Adj Close": "close", "High": "high", "Low": "low", "Volume": "volume"}, inplace=True)
    # drop close column
    df.drop("Close", inplace=True, axis=1)
    # fill weekends
    if fill_weekends:
        df = df.resample('D').ffill()
    # get all functions in finta
    finta_functions = [func for func in dir(TA) if callable(getattr(TA, func)) and not func.startswith("__")]
    # loop through all functions in finta and append the results to the dataframe
    # skip functions that throw errors
    for func in finta_functions:
        try:
            df[func] = getattr(TA, func)(df)
        except:
            pass
    # fill in missing values
    if fillna:
        df.fillna(method='bfill', inplace=True)
        df.fillna(method='ffill', inplace=True)
    # scale to standard deviation, by column
    if scale_to_std:
        df = (df - df.mean()) / df.std()
    return df

In [11]:
# function that adds sine and cosine of weekday, monthday, yearday to dataframe
# takes into account whether the data is daily, hourly, minutely, etc.
# also takes into account whether data includes weekends or not
# if data does not include weekends, assume the week is 5 days, not 7, month is 21 days, not 31, and year is 250 days, not 365
def add_time_features(df):
    """
    Adds sine and cosine of weekday, monthday, yearday to dataframe
    :param df: dataframe to add time features to
    :return: dataframe with time features
    """
    # get frequency of data
    freq = pd.infer_freq(df.index)
    # if frequency is daily, assume data includes weekends
    if freq == 'D':
        include_weekends = True
    else:
        include_weekends = False

    # get number of days in week, month, year
    if include_weekends:
        days_in_week = 7
        days_in_month = 31
        days_in_year = 365
    else:
        days_in_week = 5
        days_in_month = 21
        days_in_year = 250
    # add weekday, monthday, yearday features
    df['weekday'] = df.index.dayofweek
    df['monthday'] = df.index.day
    df['yearday'] = df.index.dayofyear
    # add sine and cosine of weekday, monthday, yearday features
    df['sin_weekday'] = np.sin(2 * np.pi * df['weekday'] / days_in_week)
    df['cos_weekday'] = np.cos(2 * np.pi * df['weekday'] / days_in_week)
    df['sin_monthday'] = np.sin(2 * np.pi * df['monthday'] / days_in_month)
    df['cos_monthday'] = np.cos(2 * np.pi * df['monthday'] / days_in_month)
    df['sin_yearday'] = np.sin(2 * np.pi * df['yearday'] / days_in_year)
    df['cos_yearday'] = np.cos(2 * np.pi * df['yearday'] / days_in_year)
    # drop weekday, monthday, yearday features
    df.drop(['weekday', 'monthday', 'yearday'], inplace=True, axis=1)
    return df

In [12]:
def sliding_window(df, window_size=10):
    """
    Creates a sliding window mechanism for a given dataframe
    :param df: dataframe
    :param window_size: window size
    :return: list of windows as dataframes
    """
    windows = []
    for i in range(len(df) - window_size + 1):
        windows.append(df.iloc[i:i + window_size])
    return windows

In [13]:
sp500_df = create_ta_features()
sp500_df.head(20)

[*********************100%***********************]  1 of 1 completed


C:\Users\Maciej\anaconda3\envs\pytorch_gpu\lib\site-packages\finta\finta.py:399: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for x, y in zip(x.fillna(0).iteritems(), y.iteritems()):
C:\Users\Maciej\anaconda3\envs\pytorch_gpu\lib\site-packages\finta\finta.py:399: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for x, y in zip(x.fillna(0).iteritems(), y.iteritems()):
C:\Users\Maciej\anaconda3\envs\pytorch_gpu\lib\site-packages\finta\finta.py:399: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for x, y in zip(x.fillna(0).iteritems(), y.iteritems()):
C:\Users\Maciej\anaconda3\envs\pytorch_gpu\lib\site-packages\finta\finta.py:399: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for x, y in zip(x.fillna(0).iteritems(), y.iteritems()):
C:\Users\Maciej\anaconda3\envs\pytorch_g

,open,high,low,close,volume,ADL,ADX,AO,ATR,BBWIDTH,...,VAMA,VBM,VFI,VPT,VWAP,VZO,WILLIAMS,WMA,WOBV,ZLEMA
Date,,,,,,,,,,,,,,,,,,,,,
2010-01-04,-1.299113,-1.286708,-1.293664,-1.282928,0.049746,-1.833595,6.432831,-0.703535,-0.730905,0.147278,...,-1.274752,0.604053,-0.437061,-1.758947,-1.302675,-0.247649,-0.179769,-1.272791,-1.100389,-1.275609
2010-01-05,-1.282826,-1.283933,-1.280318,-1.279357,-1.377868,-1.830133,6.432831,-0.703535,-0.730905,0.147278,...,-1.274752,0.604053,-0.437061,-1.755258,-1.296173,1.659163,-0.179769,-1.272791,-1.100389,-1.275609
2010-01-06,-1.279741,-1.281359,-1.275947,-1.278730,0.983417,-1.828580,6.432831,-0.703535,-0.730905,0.147278,...,-1.274752,0.604053,-0.437061,-1.751630,-1.288821,2.959055,-0.179769,-1.272791,-1.098638,-1.275609
2010-01-07,-1.279174,-1.278071,-1.278627,-1.274127,1.266984,-1.822062,6.432831,-0.703535,-0.730905,0.147278,...,-1.274752,0.604053,-0.437061,-1.744774,-1.284374,3.464771,-0.179769,-1.272791,-1.085015,-1.275609
2010-01-08,-1.274875,-1.275126,-1.273635,-1.270798,0.428625,-1.816325,6.432831,-0.703535,-0.730905,0.147278,...,-1.274752,0.604053,-0.437061,-1.739065,-1.280347,3.687970,-0.179769,-1.272791,-1.076811,-1.275609
2010-01-09,-1.274875,-1.275126,-1.273635,-1.270798,0.428625,-1.810589,6.432831,-0.703535,-0.730905,0.147278,...,-1.274752,0.604053,-0.437061,-1.733357,-1.277707,2.766435,-0.179769,-1.272791,-1.076811,-1.275609
2010-01-10,-1.274875,-1.275126,-1.273635,-1.270798,0.428625,-1.804853,6.432831,-0.703535,-0.730905,0.147278,...,-1.274752,0.604053,-0.437061,-1.727649,-1.275841,2.125317,-0.179769,-1.272791,-1.076811,-1.275609
2010-01-11,-1.269371,-1.270752,-1.267726,-1.268775,0.301304,-1.803112,6.432831,-0.703535,-0.730905,0.147278,...,-1.274752,0.604053,-0.437061,-1.726146,-1.273170,2.544684,-0.179769,-1.272791,-1.071975,-1.275609
2010-01-12,-1.271546,-1.276714,-1.278168,-1.279660,0.739357,-1.804878,6.432831,-0.703535,-0.730905,0.147278,...,-1.274752,0.604053,-0.437061,-1.734095,-1.273745,1.095195,-0.179769,-1.272791,-1.100803,-1.275609


In [14]:
financial_data_autoencoder = LSTM_AE(
  input_dim=sp500_df.shape[1],
  encoding_dim=32,
  h_dims=[128, 64],
  h_activ=None,
  out_activ=None
)
# move model to GPU
financial_data_autoencoder = financial_data_autoencoder.to('cuda')

In [15]:
training_data = torch.tensor(sp500_df.values, dtype=torch.float32, device='cuda')

In [35]:
losses = train_model(financial_data_autoencoder, training_data, epochs=100, verbose=True, lr=1e-3, denoise=False)

C:\Users\Maciej\anaconda3\envs\pytorch_gpu\lib\site-packages\torch\nn\_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
C:\Users\Maciej\anaconda3\envs\pytorch_gpu\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([66])) that is different to the input size (torch.Size([66, 66])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch: 1, Loss: 1693.452533261973
Epoch: 2, Loss: 1280.6485426807885
Epoch: 3, Loss: 1217.8714202829397
Epoch: 4, Loss: 1224.7812482985034
Epoch: 5, Loss: 1177.7293147686962
Epoch: 6, Loss: 1166.7640051688954
Epoch: 7, Loss: 1165.7868077244364
Epoch: 8, Loss: 1168.6432410653485
Epoch: 9, Loss: 1158.4890689142237
Epoch: 10, Loss: 1154.796830008444
Epoch: 11, Loss: 1147.761008187048
Epoch: 12, Loss: 1152.2500398195211
Epoch: 13, Loss: 1152.7655885030608
Epoch: 14, Loss: 1132.8372334485111
Epoch: 15, Loss: 1138.2689755360955
Epoch: 16, Loss: 1154.133754000093
Epoch: 17, Loss: 1144.8016871827047
Epoch: 18, Loss: 1142.7148389092586
Epoch: 19, Loss: 1156.1241617636979
Epoch: 20, Loss: 1146.193232332072
Epoch: 21, Loss: 1147.5817251285855
Epoch: 22, Loss: 1137.3829590281186
Epoch: 23, Loss: 1141.513501728566
Epoch: 24, Loss: 1136.6895291423316
Epoch: 25, Loss: 1137.9719301612945
Epoch: 26, Loss: 1122.084078282168
Epoch: 27, Loss: 1113.3859779911138
Epoch: 28, Loss: 1127.7870183240302
Epoch: 2

KeyboardInterrupt: 

### Defining non LSTM autoencoder just using linear layers

In [45]:
# define encoder class which will take in a sliding window of stock data 2d tensor and transform it into a 1d tensor
class LinearEncoder(nn.Module):
    def __init__(self, input_dims: Tuple[int, int], encoding_dim: int, h_dims: List[int]):
        super(LinearEncoder, self).__init__()
        # define list of hidden layers
        self.h_layers = nn.ModuleList()
        # define list of hidden layer dimensions
        self.h_dims = [input_dims[0] * input_dims[1], *h_dims, encoding_dim]
        # loop through all hidden layers
        for i in range(len(self.h_dims) - 1):
            # add linear transformation to list of hidden layers
            self.h_layers.append(nn.Linear(self.h_dims[i], self.h_dims[i + 1]))
        # define output layer
        self.out_layer = nn.Linear(self.h_dims[-1], encoding_dim)

    def forward(self, x):
        # flatten input
        x = x.view(x.shape[0], -1)
        # loop through all hidden layers
        for layer in self.h_layers:
            # apply linear transformation
            x = layer(x)
            # apply activation function
            x = nn.functional.sigmoid(x)
        # apply linear transformation to output layer
        x = self.out_layer(x)
        return x


# define decoder class which will take in a 1d tensor and transform it into a 2d tensor
class LinearDecoder(nn.Module):
    def __init__(self, input_dim: int, output_dims: Tuple[int, int], h_dims: List[int]):
        super(LinearDecoder, self).__init__()
        # define list of hidden layers
        self.h_layers = nn.ModuleList()
        self.output_dims = output_dims
        # define list of hidden layer dimensions
        self.h_dims = [input_dim, *h_dims, output_dims[0] * output_dims[1]]
        # loop through all hidden layers
        for i in range(len(self.h_dims) - 1):
            # add linear transformation to list of hidden layers
            self.h_layers.append(nn.Linear(self.h_dims[i], self.h_dims[i + 1]))
        # define output layer
        self.out_layer = nn.Linear(self.h_dims[-1], output_dims[0] * output_dims[1])

    def forward(self, x):
        # loop through all hidden layers
        for layer in self.h_layers:
            # apply linear transformation
            x = layer(x)
            # apply activation function
            x = nn.functional.sigmoid(x)
        # apply linear transformation to output layer
        x = self.out_layer(x)
        # reshape output
        x = x.view(x.size(0), *self.output_dims)
        return x


# define autoencoder class which will take in a sliding window of stock data 2d tensor
# and output a 2d tensor of the same size
class LinearAE(nn.Module):
    def __init__(self, input_dim, encoding_dim, h_dims, h_activ, out_activ):
        super(LinearAE, self).__init__()
        # define encoder
        self.encoder = LinearEncoder(input_dim, encoding_dim, h_dims)
        # define decoder
        self.decoder = LinearDecoder(encoding_dim, input_dim, h_dims)


    def forward(self, x):
        z = self.encoder(x)
        x_prime = self.decoder(z)
        return x_prime

In [46]:
linear_ae = LinearAE(
  input_dim=(30, sp500_df.shape[1]),
  encoding_dim=32,
  h_dims=[128, 64],
  h_activ=None,
  out_activ=None
)
# move model to GPU
linear_ae = linear_ae.to('cuda')
# get sample data
sample_data = sp500_df.iloc[:30].values
sample_data = torch.tensor(sample_data, dtype=torch.float32, device='cuda').resize(1, 30, sp500_df.shape[1])

# get output of model
output = linear_ae(sample_data)

C:\Users\Maciej\anaconda3\envs\pytorch_gpu\lib\site-packages\torch\_tensor.py:760: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")
C:\Users\Maciej\anaconda3\envs\pytorch_gpu\lib\site-packages\torch\nn\functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


### Defining Sliding Window Data Loader

In [47]:
from torch.utils.data import Dataset, DataLoader
class SlidingWindowDataset(Dataset):
    def __init__(self, df, window_size=10):
        self.df = df
        self.window_size = window_size

    def __len__(self):
        return self.df.shape[0] - self.window_size + 1

    def __getitem__(self, idx):
        return torch.tensor(self.df.iloc[idx:idx + self.window_size].values, dtype=torch.float32)

In [54]:
all_data_dataset = SlidingWindowDataset(sp500_df, window_size=30)

In [58]:
all_data_dataloader = DataLoader(all_data_dataset, batch_size=256, shuffle=False)

In [59]:
linear_ae = LinearAE(
  input_dim=(30, sp500_df.shape[1]),
  encoding_dim=32,
  h_dims=[512, 128, 64],
  h_activ=None,
  out_activ=None
)
# move model to GPU
linear_ae = linear_ae.to('cuda')

### Defining training loop for non LSTM autoencoder

In [61]:
epochs = 200
lr = 3e-4
optimizer = torch.optim.Adam(linear_ae.parameters(), lr=lr)
losses = []
for epoch in range(epochs):
    for batch in all_data_dataloader:
        batch = batch.to('cuda')
        output = linear_ae(batch)
        loss = nn.functional.mse_loss(output, batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss)
    print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

C:\Users\Maciej\anaconda3\envs\pytorch_gpu\lib\site-packages\torch\nn\functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 1, Loss: 2.5894477367401123
Epoch: 2, Loss: 1.9995976686477661
Epoch: 3, Loss: 1.9774116277694702
Epoch: 4, Loss: 2.0243165493011475
Epoch: 5, Loss: 2.040438413619995
Epoch: 6, Loss: 2.0385499000549316
Epoch: 7, Loss: 2.035195827484131
Epoch: 8, Loss: 2.0331673622131348
Epoch: 9, Loss: 2.031864881515503
Epoch: 10, Loss: 2.030794382095337
Epoch: 11, Loss: 2.0298213958740234
Epoch: 12, Loss: 2.02893328666687
Epoch: 13, Loss: 2.028135061264038
Epoch: 14, Loss: 2.0274240970611572
Epoch: 15, Loss: 2.026794195175171
Epoch: 16, Loss: 2.026237726211548
Epoch: 17, Loss: 2.025747060775757
Epoch: 18, Loss: 2.0253143310546875
Epoch: 19, Loss: 2.0249335765838623
Epoch: 20, Loss: 2.024599075317383
Epoch: 21, Loss: 2.0243067741394043
Epoch: 22, Loss: 2.0240495204925537
Epoch: 23, Loss: 2.02382230758667
Epoch: 24, Loss: 2.023622751235962
Epoch: 25, Loss: 2.0234479904174805
Epoch: 26, Loss: 2.0232958793640137
Epoch: 27, Loss: 2.023163318634033
Epoch: 28, Loss: 2.0230486392974854
Epoch: 29, Loss:

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB (GPU 0; 8.00 GiB total capacity; 7.21 GiB already allocated; 0 bytes free; 7.28 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF